# BatchEncoder (For BFV or BGV scheme)

In [2]:
import tenseal.sealapi as seal
import numpy as np
import util

In `1_bfv_basics.cpp` we showed how to perform a very simple computation using the BFV scheme. The computation was performed modulo the `plain_modulus` parameter, and utilized only one coefficient from a BFV plaintext polynomial. This approach has
two notable problems:

1. Practical applications typically use integer or real number arithmetic, not modular arithmetic;
2. We used only one coefficient of the plaintext polynomial. This is really wasteful, as the plaintext polynomial is large and will in any case be encrypted in its entirety.

For (1), one may ask why not just increase the `plain_modulus` parameter until no overflow occurs, and the computations behave as in integer arithmetic. The problem is that increasing `plain_modulus` increases noise budget consumption, and decreases
the initial noise budget too.

In these examples we will discuss other ways of laying out data into plaintext elements (encoding) that allow more computations without data type overflow, and can allow the full plaintext polynomial to be utilized.

Let N denote the `poly_modulus_degree` and `T` denote the `plain_modulus`. Batching allows the BFV plaintext polynomials to be viewed as 2-by-(N/2) matrices, with each element an integer modulo `T`. In the matrix view, encrypted operations act element-wise on encrypted matrices, allowing the user to obtain speeds-ups of several orders of magnitude in fully vectorizable computations. Thus, in all but the simplest computations, batching should be the preferred method to use with BFV, and when used properly will result in implementations outperforming anything done without batching.

In a later example, we will demonstrate how to use the BGV scheme. Batching works similarly for the BGV scheme to this example for the BFV scheme. For example, simply changing `scheme_type::bfv` into `scheme_type::bgv` can make this example work for the BGV scheme.

In [3]:
parms = seal.EncryptionParameters(seal.SCHEME_TYPE.BFV)
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(seal.CoeffModulus.BFVDefault(poly_modulus_degree, seal.SEC_LEVEL_TYPE.TC128))

To enable batching, we need to set the `plain_modulus` to be a prime number congruent to `1 modulo 2*poly_modulus_degree`. Microsoft SEAL provides a helper method for finding such a prime. In this example we create a 20-bit prime that supports batching.

In [4]:
parms.set_plain_modulus(seal.PlainModulus.Batching(poly_modulus_degree, 20))

context = seal.SEALContext(parms, True, seal.SEC_LEVEL_TYPE.TC128)
util.print_parameters(context)

/
|Encryption parameters: 
|	scheme: BFV
|	poly_modulus_degree: 8192
|	coeff_modulus_size: 218 (43 43 44 44 44) bits
|	plain_modulus: 1032193


We can verify that batching is indeed enabled by looking at the encryption parameter qualifiers created by `SEALContext`.

In [5]:
qualifiers = context.first_context_data().qualifiers()
print('Batching enabled: ' + str(qualifiers.using_batching))
    
keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key()
public_key = seal.PublicKey()
keygen.create_public_key(public_key)
relin_keys = seal.RelinKeys()
keygen.create_relin_keys(relin_keys)
encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)

Batching enabled: True


Batching is done through an instance of the `BatchEncoder` class.

The total number of batching `slots` equals the `poly_modulus_degree`, N, and these slots are organized into 2-by-(N/2) matrices that can be encrypted and computed on. Each slot contains an `integer modulo plain_modulus`.

In [6]:
batch_encoder = seal.BatchEncoder(context)
slot_count = batch_encoder.slot_count()
row_size = int(slot_count/2)
print('Plaintext matrix row size: ' + str(row_size))

Plaintext matrix row size: 4096


The matrix plaintext is simply given to BatchEncoder as a flattened vector of numbers. The first `row_size' many numbers form the first row, and the rest form the second row. Here we create the following matrix:

    [ 0,  1,  2,  3,  0,  0, ...,  0 ]
    [ 4,  5,  6,  7,  0,  0, ...,  0 ]

In [7]:
pod_matrix = np.zeros((slot_count,), dtype='uint64')
pod_matrix[0:4] = [0, 1, 2, 3]
pod_matrix[row_size:row_size+4] = [4, 5, 6, 7]
print('Input plaintext matrix: ')
print(pod_matrix)

Input plaintext matrix: 
[0 1 2 ... 0 0 0]


First we use `BatchEncoder` to encode the matrix into a plaintext polynomial.

In [8]:
plain_matrix = seal.Plaintext()
batch_encoder.encode(pod_matrix.tolist(), plain_matrix)

We can instantly decode to verify correctness of the encoding. Note that no
encryption or decryption has yet taken place.

In [9]:
pod_result = batch_encoder.decode_uint64(plain_matrix)
print(pod_result[0:8])                     # Slicing to avoid printing the whole 4096 slots
print(pod_result[row_size:row_size+8])

[0, 1, 2, 3, 0, 0, 0, 0]
[4, 5, 6, 7, 0, 0, 0, 0]


Next we encrypt the encoded plaintext.

In [10]:
encrypted_matrix = seal.Ciphertext()
encryptor.encrypt(plain_matrix, encrypted_matrix)
print('Noise budged in encrypted matrix: ' + str(decryptor.invariant_noise_budget(encrypted_matrix)))


Noise budged in encrypted matrix: 146


Operating on the ciphertext results in homomorphic operations being performed
simultaneously in all 8192 slots (matrix elements). To illustrate this, we
form another plaintext matrix

    [ 1,  2,  1,  2,  1,  2, ..., 2 ]
    [ 1,  2,  1,  2,  1,  2, ..., 2 ]
    
and encode it into a plaintext.

In [11]:
pod_matrix2 = np.ones((slot_count,), dtype='uint64')
pod_matrix2[1:slot_count:2] = 2
plain_matrix2 = seal.Plaintext()
batch_encoder.encode(pod_matrix2.tolist(), plain_matrix2)
print('Second input plaintext matrix: ')
print(pod_matrix2)

Second input plaintext matrix: 
[1 2 1 ... 2 1 2]


We now add the second (plaintext) matrix to the encrypted matrix, and square
the sum.

In [12]:
evaluator.add_plain_inplace(encrypted_matrix, plain_matrix2)
evaluator.square_inplace(encrypted_matrix)
evaluator.relinearize_inplace(encrypted_matrix, relin_keys)

How much noise budget do we have left?

In [13]:
print('Noise budget in result: ' + str(decryptor.invariant_noise_budget(encrypted_matrix)))

Noise budget in result: 114


We decrypt and decompose the plaintext to recover the result as a matrix.

In [16]:
plain_result = seal.Plaintext()
decryptor.decrypt(encrypted_matrix, plain_result)
pod_result = batch_encoder.decode_uint64(plain_result)
print('Result plaintext matrix:')
print(pod_result[0:8])
print(pod_result[row_size:row_size+8])

Result plaintext matrix:
[1, 9, 9, 25, 1, 4, 1, 4]
[25, 49, 49, 81, 1, 4, 1, 4]


Batching allows us to efficiently use the full plaintext polynomial when the desired encrypted computation is highly parallelizable. However, it has not solved the other problem mentioned in the beginning of this file: each slot holds only an integer modulo `plain_modulus`, and unless `plain_modulus` is very large, we can quickly encounter data type overflow and get unexpected results when integer computations are desired. Note that overflow cannot be detected in encrypted form. The CKKS scheme (and the `CKKSEncoder`) addresses the data type overflow issue, but at the cost of yielding only approximate results.